In [ ]:
get_ipython().system('curl -L -o ./sql-injection-dataset1.zip   https://www.kaggle.com/api/v1/datasets/download/syedsaqlainhussain/sql-injection-dataset')
#get_ipython().system('curl -L -o ./sql-injection-dataset.zip   https://www.kaggle.com/api/v1/datasets/download/sajid576/sql-injection-dataset')

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 1169k  100 1169k    0     0  2168k      0 --:--:-- --:--:-- --:--:-- 2168k


In [ ]:
get_ipython().system('unzip ./sql-injection-dataset1.zip -d sql-injection-dataset')
#get_ipython().system('unzip ./sql-injection-dataset.zip -d sql-injection-dataset')

Archive:  ./sql-injection-dataset1.zip
  inflating: sql-injection-dataset/SQLiV3.csv  
  inflating: sql-injection-dataset/sqli.csv  
  inflating: sql-injection-dataset/sqliv2.csv  


===

In [ ]:
import pandas as pd
import chardet

with open("sql-injection-dataset/sqliv2.csv", "rb") as f:
    raw = f.read(100000)  # read first 100KB
    encoding = chardet.detect(raw)['encoding']
    print("Detected encoding:", encoding)

df = pd.read_csv("/content/sql-injection-dataset/sqliv2.csv", encoding=encoding, on_bad_lines='skip')

Detected encoding: UTF-16


In [ ]:
df.head()

,Sentence,Label
0,NaN,1
1,""" or pg_sleep ( __TIME__ ) --",1
2,create user name identified by pass123 tempora...,1
3,%29,1
4,' AND 1 = utl_inaddr.get_host_address ( ( S...,1


In [ ]:
rows, cols = df.shape
print(f"Number of rows: {rows}")
print(f"Number of columns: {cols}")

Number of rows: 33761
Number of columns: 2


In [ ]:
import pandas as pd
import numpy as np
import pickle
import re
import math
import os
from scipy.sparse import hstack, csr_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report
from xgboost import XGBClassifier

# ============================================================
# 1. Load the dataset with correct encoding

df = df.rename(columns={"Sentence": "query", "Label": "label"})
df["query"] = df["query"].astype(str).str.strip()
df = df.dropna(subset=["query", "label"])
df["label"] = df["label"].astype(int)

print(f"Dataset loaded: {df.shape[0]} rows")
print(df["label"].value_counts())

# ============================================================
# 2. Feature Engineering

def extract_features(df):
    q = df["query"]
    return pd.DataFrame({
        "length": q.str.len(),
        "digits_ratio": q.str.count(r"\d") / (q.str.len() + 1),
        "special_ratio": q.str.count(r"[^a-zA-Z0-9\s]") / (q.str.len() + 1),
        "uppercase_ratio": q.str.count(r"[A-Z]") / (q.str.len() + 1),
        "has_semicolon": q.str.contains(";").astype(int),
        "has_comment": q.str.contains("--|#|/\*").astype(int),
        "has_quotes": q.str.count(r"['\"]"),
        "sql_keywords": q.str.count(r"\b(select|union|insert|drop|delete|update|from|where|or|and)\b", flags=re.IGNORECASE),
        "tool_keywords": q.str.count(r"\b(sqlmap|nmap|metasploit|burp|hydra|nikto|exploit|shodan)\b", flags=re.IGNORECASE),
        "entropy": q.apply(lambda x: 0 if len(x)==0 else -sum(p*math.log2(p) for p in (x.count(c)/len(x) for c in set(x)) if p>0))
    })

# Malicious-only character n-gram TF-IDF (key for catching obfuscated payloads)
mal_tfidf = TfidfVectorizer(analyzer='char', ngram_range=(3,6), lowercase=True, max_features=5000)
mal_tfidf.fit(df[df["label"]==1]["query"])

# ============================================================
# 3. Train or Load Model


model_path = "sql_search_detector.pkl"

if not os.path.exists(model_path):
    print("Training model (this takes ~30–60 seconds)...")

    tfidf_X = mal_tfidf.transform(df["query"])
    hand_X = extract_features(df)
    X = hstack([tfidf_X, csr_matrix(hand_X.values)])
    y = df["label"]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

    scale_pos = (len(y_train) - y_train.sum()) / y_train.sum()

    model = XGBClassifier(
        n_estimators=600,
        max_depth=10,
        learning_rate=0.05,
        scale_pos_weight=scale_pos,
        random_state=42,
        n_jobs=-1,
        eval_metric="logloss"
    )
    model.fit(X_train, y_train)

    preds = model.predict(X_test)
    probs = model.predict_proba(X_test)[:,1]

    print("=== Model Performance ===")
    print(f"Accuracy : {accuracy_score(y_test, preds):.4f}")
    print(f"ROC-AUC  : {roc_auc_score(y_test, probs):.4f}")
    print(classification_report(y_test, preds))

    with open(model_path, "wb") as f:
        pickle.dump((model, mal_tfidf, hand_X.columns.tolist()), f)
    print("Model saved.")
else:
    print("Loading pre-trained model...")
    with open(model_path, "rb") as f:
        model, mal_tfidf, hand_cols = pickle.load(f)

# ============================================================
# 4. Prediction Function

def predict_query(text):
    tfidf_vec = mal_tfidf.transform([text])
    hand_df = extract_features(pd.DataFrame({"query": [text]}))
    hand_vec = csr_matrix(hand_df.values)
    X = hstack([tfidf_vec, hand_vec])

    prob_mal = model.predict_proba(X)[0][1]
    label = "BAD (potentially malicious / attack-related)" if prob_mal > 0.5 else "GOOD (normal search query)"
    confidence = round(max(prob_mal, 1-prob_mal) * 100, 2)
    mal_percent = round(prob_mal * 100, 2)

    return label, confidence, mal_percent

# ============================================================
# 5. Interactive Testing

print("\n" + "="*60)
print("   Malicious Search Query Detector – Ready")
print("="*60 + "\n")

while True:
    q = input("Enter search query (or 'quit'): ").strip()
    if q.lower() in ["quit", "exit", "q"]:
        break
    if not q:
        continue

    result, conf, mal_prob = predict_query(q)
    print(f"\n→ Result: {result}")
    print(f"   Confidence       : {conf}%")
    print(f"   Malicious probability : {mal_prob}%")
    print("-" * 55)

<>:35: SyntaxWarning: invalid escape sequence '\*'
<>:35: SyntaxWarning: invalid escape sequence '\*'
/tmp/ipython-input-2967842336.py:35: SyntaxWarning: invalid escape sequence '\*'
  "has_comment": q.str.contains("--|#|/\*").astype(int),


Dataset loaded: 33761 rows
label
0    22305
1    11456
Name: count, dtype: int64
Training model (this takes ~30–60 seconds)...
=== Model Performance ===
Accuracy : 0.9981
ROC-AUC  : 0.9998
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      4462
           1       1.00      1.00      1.00      2291

    accuracy                           1.00      6753
   macro avg       1.00      1.00      1.00      6753
weighted avg       1.00      1.00      1.00      6753

Model saved.

   Malicious Search Query Detector – Ready


→ Result: BAD (potentially malicious / attack-related)
   Confidence       : 99.97000122070312%
   Malicious probability : 99.97000122070312%
-------------------------------------------------------
